# Llama-2-Chat Models from Hugging Face

This notebook illustrates Hugging Face version of Llama 2 Chat models used as LangChain's Chat model.

Using the Hugging Face Llama 2 Chat models integration has several benefits:
- **Flexibility in setting model inference parameters:** For instance, it is possible to define the HF Pipeline with the `meta-llama/Llama-2-7b-chat-hf` model and `BitsAndBytesConfig` that loads the model in 4-bits, and run it locally on a GPU with less than 6GB of VRAM.
- **Automatic prompt formatting that adheres to Meta's guidelines:** The `ChatLlama2Hf` class includes the `format_messages_as_text method`, which converts LangChain's messages into prompts that comply with [Meta's guidelines](https://huggingface.co/blog/llama2) for interacting with Llama 2 models.
- **Customization:** By overwriting static method `format_messages_as_text` one can also use other instruction-tuned models with LangChain's "chat messages" interface.

## Setup

In [6]:
# Hugging Face imports:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)

# LangChain imports:
from langchain.chat_models import ChatLlama2Hf
from langchain.schema import AIMessage, HumanMessage, SystemMessage

This notebook assumes that you were granted with access to the Llama 2 models in the Hugging Face models hub. To use the model locally, you need to be [logged in](https://huggingface.co/docs/huggingface_hub/quick-start#login) with a Hugging Face account. 

To log in using CLI run the following command in your terminal:
```
huggingface-cli login
```
or using an environment variable
```
huggingface-cli login --token $HUGGINGFACE_TOKEN
```

You can also log in programmatically in notebook:

In [7]:
from huggingface_hub import login
login()

## Creating Hugging Face Pipeline instance:

The following section loads the 7b version of the Llama 2 Chat model and uses the `bitsandbytes` library to load a model in 4bit using NF4 quantization with double quantization and compute dtype bfloat16, which speeds up the underlying matrix multiplications.

More information about these techniques can be found at: [link](https://huggingface.co/blog/4bit-transformers-bitsandbytes)

In [8]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

To load the model in 4bit, make sure that the `accelerate`, `transformers` and `bitsandbytes` libraries are installed:

In [9]:
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_4bit = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
pipe = pipeline(
    "text-generation",
    model=model_4bit,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

Initializing a LangChain Llama-2-Chat instance

In [13]:
chat = ChatLlama2Hf(pipeline=pipe)

Besides defining arguments for `Pipeline` initialization, we can also control the generation process, by enabling sampling, chaning temperature or defining maximum length of single generation.

In [14]:
# Generation kwargs:
pipeline_kwargs = {
    "do_sample": True,
    "top_p": 0.95,
    "temperature": 0.7,
    "eos_token_id": tokenizer.eos_token_id,
    "max_length": 512,    
}

### Single calls:

You can get chat completions by passing one or more messages to the chat model. The response will be a message:

In [15]:
messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to French."
    ),
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    ),
]
result = chat(messages, **pipeline_kwargs)
print(result.content)

 Sure! Here is the translation of "I love programming" from English to French:

Je adore le programming.

I hope this helps! Let me know if you have any other sentences you would like me to translate.


### Single calls with stop words

By utilizing Hugging Face [Stopping Criteria](https://huggingface.co/docs/transformers/internal/generation_utils#transformers.StoppingCriteria) under the hood, we can provide phrases that, if generated by the model, will cause the generation process to stop.

In [16]:
messages = [
    SystemMessage(
        content="You are a helpful assistant."
    ),
    HumanMessage(
        content="Tell me the history of AI."
    ),
]
result = chat(messages, stop=["Artificial", "Inteligence"], **pipeline_kwargs)
print(result.content)

 Of course, I'd be happy to help! The history of Artificial


### Batch calls:

We can also go one step further and generate completions for multiple sets of messages:

In [17]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love artificial intelligence.")
    ],
]
result = chat.generate(batch_messages)

In [18]:
for i, generation in enumerate(result.generations):
    print(f"Response #{i}:\n{generation[0].text}", end="\n\n")

Response #0:
 Great! "Programmation" is the French word for "programming". So, "J'aime le programmation" (pronounced "zhem-ahy lah-pree-moh").
Would you like me to translate anything else for you?

Response #1:
 Bonjour! Je suis heureux d'être votre assistant de traduction pour l'anglais à français.
Vous aimez l'intelligence artificielle? C'est une technologie très intéressante, n'est-ce pas? Elle permet de automatiser de nombreux processus et de faire des choses qui auraient autrement été très difficiles ou impossibles.
Si vous avez d'autres questions ou des phrases à traduire, n'hésitez pas à me demander de l'aide. Je suis là pour vous aider!

